## Dataset Creation

In this notebook, we will create the actual dataset that we will train our model on. In particular, we will:
1. Select the features we want to train our model on.
2. Specify how the features should be preprocessed.
3. Create a dataset split for training and validation data.

![tutorial-flow](images/create_training_dataset.png)

In [1]:
import hsfs

conn = hsfs.connection()
fs = conn.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


### Feature Selection

We start by selecting all the features we want to include for model training/inference.

In [2]:
# Load feature groups.
trans_fg = fs.get_feature_group("transactions", 1)
window_aggs_fg = fs.get_feature_group("transactions_4h_aggs", 1)

# Select features for training data.
ds_query = trans_fg.select(["fraud_label", "category", "amount", "age_at_transaction", "days_until_card_expires", "loc_delta"])\
    .join(window_aggs_fg.select_except(["cc_num"]), on="cc_num")\

ds_query.show(5)

2022-05-24 17:47:33,994 INFO: USE `fv_test_featurestore`
2022-05-24 17:47:34,882 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`fraud_label` `fraud_label`, `fg1`.`category` `category`, `fg1`.`amount` `amount`, `fg1`.`age_at_transaction` `age_at_transaction`, `fg1`.`days_until_card_expires` `days_until_card_expires`, `fg1`.`loc_delta` `loc_delta`, `fg1`.`cc_num` `join_pk_cc_num`, `fg1`.`datetime` `join_evt_datetime`, `fg0`.`trans_volume_mstd` `trans_volume_mstd`, `fg0`.`trans_volume_mavg` `trans_volume_mavg`, `fg0`.`trans_freq` `trans_freq`, `fg0`.`loc_delta_mavg` `loc_delta_mavg`, RANK() OVER (PARTITION BY `fg1`.`cc_num`, `fg1`.`datetime` ORDER BY `fg0`.`datetime` DESC) pit_rank_hopsworks
FROM `fv_test_featurestore`.`transactions_1` `fg1`
INNER JOIN `fv_test_featurestore`.`transactions_4h_aggs_1` `fg0` ON `fg1`.`cc_num` = `fg0`.`cc_num` AND `fg1`.`datetime` >= `fg0`.`datetime`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`fraud_label` `fraud_label`, `right_fg0`.`categ

,fraud_label,category,amount,age_at_transaction,days_until_card_expires,loc_delta,trans_volume_mstd,trans_volume_mavg,trans_freq,loc_delta_mavg
0,0,Grocery,93.51,25.334094,175.912280,0.000000,93.510,93.510,93.510,0.000000
1,0,Domestic Transport,65.14,25.335632,175.350486,0.319574,65.140,65.140,65.140,0.319574
2,0,Grocery,0.26,25.336235,175.130347,0.314148,0.260,0.260,0.260,0.314148
3,0,Grocery,1.43,25.336660,174.975058,0.000000,0.845,0.845,0.845,0.157074
4,0,Grocery,19.75,25.344710,172.034664,0.105313,19.750,19.750,19.750,0.105313


Recall that we computed the features in `transactions_4h_aggs` using 4-hour aggregates. If we had created multiple feature groups with identical schema for different window lengths, and wanted to include them in the join we would need to include a prefix argument in the join to avoid feature name clash. See the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/query_api/#join) for more details.

### Transformation Functions
Transformation functions are a mathematical mapping of input data that may be stateful - requiring statistics from the partent feature view (such as number of instances of a category, or mean value of a numerical feature)

We will preprocess our data using *min-max scaling* on numerical features and *label encoding* on categorical features. To do this we simply define a mapping between our features and transformation functions. This ensures that transformation functions such as *min-max scaling* are fitted only on the training data (and not the validation/test data), which ensures that there is no data leakage.

In [3]:
# Load transformation functions.
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")
label_encoder = fs.get_transformation_function(name="label_encoder")

# Map features to transformations.
transformation_functions = {
    "category": label_encoder,
    "amount": min_max_scaler,
    "trans_volume_mavg": min_max_scaler,
    "trans_volume_mstd": min_max_scaler,
    "trans_freq": min_max_scaler,
    "loc_delta": min_max_scaler,
    "loc_delta_mavg": min_max_scaler,
    "age_at_transaction": min_max_scaler,
    "days_until_card_expires": min_max_scaler,
}

### Feature View Creation

Feature view are a logical view over features in feature groups, with support for associating a label with a feature, and transformation functions with features. 

In [4]:
feature_view = fs.create_feature_view(
    name='transactions_view',
    query=ds_query,
    label=["fraud_label"],
    transformation_functions=transformation_functions
)

To view and explore data in the feature view we can retrieve batch data using `get_batch_data()` method 

In [9]:
feature_view.get_batch_data().head(5)

2022-05-24 18:55:35,679 INFO: USE `fv_test_featurestore`
2022-05-24 18:55:36,543 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`category` `category`, `fg1`.`amount` `amount`, `fg1`.`age_at_transaction` `age_at_transaction`, `fg1`.`days_until_card_expires` `days_until_card_expires`, `fg1`.`loc_delta` `loc_delta`, `fg1`.`cc_num` `join_pk_cc_num`, `fg1`.`datetime` `join_evt_datetime`, `fg0`.`trans_volume_mstd` `trans_volume_mstd`, `fg0`.`trans_volume_mavg` `trans_volume_mavg`, `fg0`.`trans_freq` `trans_freq`, `fg0`.`loc_delta_mavg` `loc_delta_mavg`, RANK() OVER (PARTITION BY `fg1`.`cc_num`, `fg1`.`datetime` ORDER BY `fg0`.`datetime` DESC) pit_rank_hopsworks
FROM `fv_test_featurestore`.`transactions_1` `fg1`
INNER JOIN `fv_test_featurestore`.`transactions_4h_aggs_1` `fg0` ON `fg1`.`cc_num` = `fg0`.`cc_num` AND `fg1`.`datetime` >= `fg0`.`datetime`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`category` `category`, `right_fg0`.`amount` `amount`, `right_fg0`.`age_at_transact

,category,amount,age_at_transaction,days_until_card_expires,loc_delta,trans_volume_mstd,trans_volume_mavg,trans_freq,loc_delta_mavg
0,Grocery,93.51,25.334094,175.912280,0.000000,93.510,93.510,93.510,0.000000
1,Domestic Transport,65.14,25.335632,175.350486,0.319574,65.140,65.140,65.140,0.319574
2,Grocery,0.26,25.336235,175.130347,0.314148,0.260,0.260,0.260,0.314148
3,Grocery,1.43,25.336660,174.975058,0.000000,0.845,0.845,0.845,0.157074
4,Grocery,19.75,25.344710,172.034664,0.105313,19.750,19.750,19.750,0.105313


#### Training Dataset Creation

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

Training Dataset  may contain splits such as: 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

Training dataset is created using `fs.create_training_dataset()` method.

In [2]:
td_version, td_job = feature_view.create_training_dataset(
    description = 'transactions_dataset_splitted',
    data_format = 'csv',
    splits = {'train': 80, 'validation': 20},
    train_split = "train",
    write_options = {'wait_for_job': True},
    coalesce = True
)

Training dataset job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/119/jobs/named/transactions_view_1_1_create_fv_td_24052022184211/executions


#### Training Dataset retreival
To retrieve training data from storage (already materialised) or from feature groups direcly we can use `get_training_dataset_splits` or `get_training_dataset` methods. If version is not provided or provided version has not already existed, it creates a new version of training data according to given arguments and returns a dataframe. If version is provided and has already existed, it reads training data from storage or feature groups and returns a dataframe. If split is provided, it reads the specific split.

In [3]:
td_version, train_df = feature_view.get_training_dataset_splits({'train': 80}, start_time=None, end_time=None, version = td_version)

In [4]:
train_df

{'train':        fraud_label  category    amount  age_at_transaction  \
 0                0         0  0.000000            0.010738   
 1                0         0  0.000000            0.047263   
 2                0         0  0.000000            0.063646   
 3                0         0  0.000000            0.340523   
 4                0         0  0.000000            0.954656   
 ...            ...       ...       ...                 ...   
 84632            1         5  0.007304            0.909320   
 84633            1         5  0.013629            0.516229   
 84634            1         8  0.000170            0.132060   
 84635            1         8  0.000488            0.488922   
 84636            1         8  0.003099            0.266807   
 
        days_until_card_expires  loc_delta  trans_volume_mstd  \
 0                     0.850530   0.024955           0.000000   
 1                     0.943808   0.035718           0.000000   
 2                     0.132038   0.00

In [5]:
td_version, val_df = feature_view.get_training_dataset_splits({'validation': 20}, start_time=None, end_time=None, version = td_version)

In [6]:
val_df

{'validation':        fraud_label  category        amount  age_at_transaction  \
 0                0         0  3.336858e-07            0.961833   
 1                0         0  6.673716e-07            0.466099   
 2                0         0  6.673716e-07            0.501453   
 3                0         0  6.673716e-07            0.659576   
 4                0         0  6.673716e-07            0.827305   
 ...            ...       ...           ...                 ...   
 21378            1         4  2.247374e-03            0.677929   
 21379            1         4  2.690842e-03            0.206518   
 21380            1         4  3.267118e-03            0.909320   
 21381            1         5  4.428011e-03            0.922000   
 21382            1         5  5.702691e-03            0.206518   
 
        days_until_card_expires  loc_delta  trans_volume_mstd  \
 0                     0.766848   0.185500       1.365109e-03   
 1                     0.924525   0.089776       6

In [7]:
td_version, df = feature_view.get_training_dataset_splits({'train': 80, 'validation': 20}, start_time=None, end_time=None, version = td_version)

In [8]:
df

{'train':        fraud_label  category    amount  age_at_transaction  \
 0                0         0  0.000000            0.010738   
 1                0         0  0.000000            0.047263   
 2                0         0  0.000000            0.063646   
 3                0         0  0.000000            0.340523   
 4                0         0  0.000000            0.954656   
 ...            ...       ...       ...                 ...   
 84632            1         5  0.007304            0.909320   
 84633            1         5  0.013629            0.516229   
 84634            1         8  0.000170            0.132060   
 84635            1         8  0.000488            0.488922   
 84636            1         8  0.003099            0.266807   
 
        days_until_card_expires  loc_delta  trans_volume_mstd  \
 0                     0.850530   0.024955           0.000000   
 1                     0.943808   0.035718           0.000000   
 2                     0.132038   0.00

## From feature view we can also retrieve feature vecors from online store for low latency model serving 
Training data version is required for transformation. Call `feature_view.init_serving(version)` to pass the training dataset version. 

In [15]:
feature_view.init_serving(td_version)

### Retrieve single feature vector

In [11]:
feature_view.get_feature_vector({"cc_num": "4473593503484549"})

[0,
 0.0031516625395292554,
 0.9651877430221095,
 0.7179505149718097,
 4.6053745689962795e-05,
 0.0031516625395292554,
 0.0031516625395292554,
 0.0031516625395292554,
 4.962147746344275e-05]

### Retrieve batch of feature vectors

Here we have to let feature_view that we need to get batch of feature vectors by passing `batch=True` to `init_serving` method  

In [17]:
feature_view.init_serving(td_version, batch=True)
card_ids = [
    "4473593503484549",
    "4336399961348201",
    "4219785543443381",
    "4137709749259770",
    "4573366597272313",
    "4929411498746287",
    "4855787436134696"    
]
feature_view.get_feature_vectors(entry={"cc_num": card_ids})

[[4,
  0.002657473844871466,
  0.20621260449599255,
  0.08422812335515834,
  0.015244317743686607,
  0.002837664185935076,
  0.002837664185935076,
  0.002837664185935076,
  0.10358870705000209],
 [0,
  0.010312893853540624,
  0.9201764105420973,
  0.3617157360201056,
  1.5696794982265385e-05,
  0.010312893853540624,
  0.010312893853540624,
  0.010312893853540624,
  1.691280799841865e-05],
 [2,
  0.003054559966844977,
  0.1260833208082077,
  0.6789481436010685,
  0.04029573683352209,
  0.003054559966844977,
  0.003054559966844977,
  0.003054559966844977,
  0.04341740215092864],
 [0,
  0.0031516625395292554,
  0.9651877430221095,
  0.7179505149718097,
  4.6053745689962795e-05,
  0.0031516625395292554,
  0.0031516625395292554,
  0.0031516625395292554,
  4.962147746344275e-05],
 [0,
  0.002340472318926226,
  0.3446710725993161,
  0.39386150765299255,
  0.06939270102016566,
  0.0026661496761078616,
  0.0026661496761078616,
  0.0026661496761078616,
  0.24965591627238473],
 [0,
  0.0028430031

### Next Steps

In the next notebook, we will train a model on the dataset we created in this notebook.